# How to securely access resources via Tailscale

[Tailscale](https://tailscale.com/) is a tool for setting up private end-to-end encrypted networks that securely connect users to services and devices. It is pretty easy to set up, has a very generous free tier (up to 3 users and up to 100 devices), and has excellent cross-platform support (so you can connect devices running Windows, MacOS, Linux, iOS, or Android).

## How I use Tailscale

I mainly use tailscale to `ssh` into my homelab server and access (via browser) applications running on my homelab server. Tailscale does some heavy lifting [under the hood](https://tailscale.com/blog/how-tailscale-works) to make this process nearly frictionless.

Before tailscale, if I wanted to `ssh` into a machine on my home network (ie a homelab) while I was away from home, it was a [complex chore](#manual-ssh) that would involve knowing your home network's public IP address, defending against dynamic IP reassignment (or procuring a static IP address from your ISP), and manually managing `ssh` hardening.
And if I wanted to use my phone to remotely access a service running on a machine on my home network, I'd have to figure that out (the utility-to-work ratio seemed so bad that I never entertained trying).

With tailscale, after a little setup, to `ssh` into a machine on my home network, I just have to:
1. enter command `tailscale ssh homelab-name` and
2. authenticate with tailscale via returned URL
then I get a shell on my homelab.

To connect to a homelab service from my phone, it's even easier. If the homelab device's name (in tailscale) is `matt-lab` and the service is running on port `8080`, I can access the service via a browser on my phone by going to `matt-lab:8080`.

![accessing from a phone](imgs/accessing_a_homelab_service_from_my_phone.png){fig-align="center" width="50%"}

# Overview of steps to set up a Tailscale Tailnet

* [Create a tailscale account](https://login.tailscale.com/start).
    * Optional: [Invite](https://tailscale.com/kb/1017/install#invite-users) other users to your tailnet.
* Install the tailscale client on each device you want to connect in the network.
    * [Linux (Ubuntu/Debian) install instructions](#linux_install)
    * [MacOS install instructions](#macos_install)
    * [Windows install instructions](#windows_install)
    * [iOS install instructions](#ios_install)
    * [Android install instructions](#android_install)
* [Configure your tailnet's ACLs](#config_acl) (Access Control Lists) to control which devices users can access.
    * define groups, specify which devices each group should be able to access, and add users to groups.
* [Configure a Homelab](#config_homelab) to accept incoming `ssh` connections

## Tailscale Client Installation on a MacOS Machine {#macos_install}

To be able to use tailscale ssh from a macOS machine, you have install the open source `tailscaled` variant per [these instructions](https://github.com/tailscale/tailscale/wiki/Tailscaled-on-macOS).

If `golang` isn't installed, [download](https://go.dev/dl/) and install golang and make sure the `go` binary is in a dir on the system's `PATH`.

Then, run these commands
```console
go install tailscale.com/cmd/tailscale{,d}@main
sudo $HOME/go/bin/tailscaled install-system-daemon
tailscale up
```

### Enabling MagicDNS

You might have to manually add your tailnet's DNS server IP address to your MacOS-running-machine's DNS servers. Go into system settings, search for **DNS**, and add IP address "100.100.100.100" ahead of the existing IP address (Most routers use "192.168.0.1"). If you sometimes connect to Wi-Fi and other times use ethernet, you'll probably have to do this for both connection modes.

![First caption](imgs/macos_dns_fix.png){fig-align="center" width="50%"}

## Tailscale Client Installation on a Windows Machine {#windows_install}

I don't have a current Windows device, but I remember it being pretty straightforward. The [official instructions](https://tailscale.com/kb/1022/install-windows) look straightforward (download and run an installer, then log in).

## Tailscale Client Installation on an Android Phone {#android_install}

1. Install the [android tailscale client](https://play.google.com/store/apps/details?id=com.tailscale.ipn) from the Play Store.
2. Launch the app.
    1. click **Get Started**.
    2. approve the prompt to set up a VPN connection.
3. Log in to your tailscale account.
4. Approve the device in the admin console.

## Tailscale Client Installation on an iPhone {#ios_install}

The steps should be mostly the same as on Android, except installing from the App Store rather than the Play Store. I don't have an iOS device, so I'll defer to the [official install instructions](https://tailscale.com/kb/1020/install-ios)).

| ![First caption](imgs/android_install_1.png){width="100%"} | ![Second caption](imgs/android_install_2.png){width="100%"} |
|:--:|:--:|
| 2.1. **Get Started** | 2.2. Click **OK** to allow a VPN connection |

| ![First caption](imgs/android_install_3.png){width="100%"} | ![Second caption](imgs/android_install_4.png){width="100%"} |
|:--:|:--:|
| 3. **Log in** | 3. via the method used when signing up |

| ![First caption](imgs/android_install_5.png){width="100%"} | ![Second caption](imgs/android_install_6.png){width="100%"} |
|:--:|:--:|
| 4. **Connect** the device | 4. Open up the Admin console |

Then click **Approve** (not pictured, too much to redact).

## Setting up ACLs {#config_acl}

Tailscale allows us to easily network and connect to devices, but you may not want to grant all network users access to all devices. Tailscale has a robust [Role Based Access Control](https://tailscale.com/blog/rbac-like-it-was-meant-to-be) framework that allows you to define maintainable and concise access control logic in your [ACL file](https://login.tailscale.com/admin/acls/file).

The [ACL syntax](https://tailscale.com/kb/1337/acl-syntax) supports very granular controls, but here's the core logic for the main fields:

* In **groups**: define a **group** for each role you want your network to support (e.g., admin, dev, ci, prod, etc) and specify the users in the group.

* In **tagOwners**: define a **tag** for each device grouping you want to support, and specify which users/groups are allowed to apply that tag to devices.

* In **acls**: define which users/groups (sources, or **src**) can connect to which destination devices and ports (**dst**).

* In **ssh**: define which users/groups (**src**) can `ssh` into which devices (**dst**), the **action** `ssh` take (either just **accept** the connection request or **check** the user by requiring reauthentication every 12 hours), and the username(s) that **src** members can `ssh` into.

* In **tests**: define tests that check that specified users/gropups can or cannot connect to specified device-ports.
    * These tests run whenever the ACL file is saved.

* In **sshTests**: define tests that check that specified groups can or cannot `ssh` into specified usernames on specified devices and face the correct authentication scrutiny.
    * these tests also run whenever the ACL file is saved.


### Simple ACL policy specification

This ACL policy facilitates a secure single-user tailnet. Just copy it into your [ACL file](https://login.tailscale.com/admin/acls/file) and update the sections with a "Replace ..." comment.

```json
// ACL for a small or single-user network
{
	// Define access control lists for users, groups, autogroups, tags,
	// Tailscale IP addresses, and subnet ranges.

	// Declare static groups of users. Use autogroups for all users or users with a specific role.
	"groups": {
         // replace that email with the emails associated to the tailscale accounts of "admin" users
		"group:admin": ["admin_user_email@email.com"],
	},

	// Define the tags which can be applied to devices and by which users.
	"tagOwners": {
		"tag:homelab":     ["group:admin"],
        "tag:workstation":     ["group:admin"],
		"tag:container":   ["autogroup:admin"],
	},

	"acls": [
		// Allow all connections from admin group members.
		{"action": "accept", "src": ["group:admin"], "dst": ["*:*"]},
		// Allow users to access their own devices.
		{
			"action": "accept",
			"src":    ["autogroup:member"],
			"dst":    ["autogroup:self:*"],
		},
	],

	// Define users and devices that can use Tailscale SSH.
	"ssh": [
		// Allow all users to SSH into their own devices in check mode.
		{
			"action": "check",
			"src":    ["autogroup:member"],
			"dst":    ["autogroup:self"],
			"users":  ["autogroup:nonroot"],
		},
        // Allow the user with email admin_user_email@email.com to ssh into any
        //   non-root username on any device with the homelab tag
		{
			"action": "check",
			"src":    ["admin_user_email@email.com"],
			"dst":    ["tag:homelab"],
			"users":  ["autogroup:nonroot"],
		},
        // Allow members of the admin group to ssh into the admin username on
        //   any any device with the homelab tag
		{
			"action": "check",
			"src":    ["group:admin"],
			"dst":    ["tag:homelab"],
            // replace "admin" with the username you want to ssh into on dest machine
			"users":  ["admin"],
		},
	],

	// Test access rules every time they're saved.
	"tests": [
		{
			"src":  "alice@example.com",
			"deny": [
                "tag:homelab:22",
                "tag:homelab:80",
                "tag:homelab:443",
                "tag:homelab:8080"
            ],
		},
		{
			"src":    "admin_user_email@email.com",
			"accept": [
                "tag:homelab:22",
                "tag:homelab:80",
                "tag:homelab:443",
                "tag:homelab:8080"
            ],
		},
	],

	"sshTests": [
		{
			"src":   "group:admin",
			"dst":   ["tag:homelab"],
			"check": ["group:admin"],
			"deny":  ["root"],
		},
		{
			"src":   "admin_user_email@gmail.com",
			"dst":   ["tag:homelab"],
			"check": ["group:admin"],
			"deny":  ["root"],
		},
	],
}

```

## Configure the homelab host to accept ssh connections {#config_homelab}

Open the [admin console](https://login.tailscale.com/admin/machines) then edit the ACL tags for the Homelab device.

In the **Edit ACL Tags** interface, click the **Add tags** dropdown and select the **tag:homelab** option. Note that these options were defined in the **tagOwner** section of ACL policy file.

![First caption](imgs/acl_open_tag_adding_interface.png){fig-align="center" width="80%"}

![First caption](imgs/acl_add_tag_to_homelab.png){fig-align="center" width="60%"}

Now, from a terminal on the homelab machine, run this command to direct tailscale to start up in `ssh` mode, which will persist even after rebooting.

```console
sudo tailscale up --ssh
```

Now you can `ssh` into the homelab machine from another tailscale-connected machine via this command (replace "admin" for your username on that machine and "homelab" with whatever label you've given tailscale for that device).

```console
tailscale ssh admin@homelab
```

# Conclusion

This post only scratches the surface of what you can stand up with tailscale, and still, I feel like I should have split it up across a few smaller posts.

In any case, in this post, we've covered:
* how to install tailscale on all major device operating systems,
* a basic ACL configuration and what the parts mean,
* how to configure a machine to accept incoming `ssh` connections

In future posts, I should write up how and why to build a homelab machine and how to build or configure LLM applications and serve them on locally on your tailnet.
